In [ ]:
"""
Mounting the Google drive
"""
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path = "/content/drive/MyDrive/Colab Notebooks/COEN240_TA/"
train_path = root_path + "train/"
validate_path = root_path + "validate/"

In [2]:
##DATA AUGMENTATION

In [ ]:
import numpy as np
import cv2
import os
import random

def translate_image(image, dx, dy, path, image_name):
    translation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])
    translated_image = cv2.warpAffine(image, translation_matrix, (image.shape[1], image.shape[0]))
    translated_image = face_box(translated_image)
    img_name = f'{image_name}_translated_dx{dx}_dy{dy}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and translated_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, translated_image)
    #Apply extra augmentations
    if translated_image is not None:
      rotate_image(translated_image, path, img_name)
      adjust_lighting(translated_image, path, img_name)

def rotate_image(image, path, image_name, angle=None):
    if angle is None:
      angle = random.randint(-10,10)
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols,rows))
    rotated_image = face_box(rotated_image)
    img_name = f'{image_name}_rotated_angle{angle}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and rotated_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, rotated_image)
    if rotated_image is not None:
      adjust_lighting(rotated_image, path, img_name)
      flip_image(rotated_image, path, img_name)

def scale_image(image, path, scale_factor, image_name):
    img_shrinked = cv2.resize(image, (200, 200), interpolation = cv2.INTER_AREA)
    img_shrinked = face_box(img_shrinked)
    img_name = f'{image_name}_shrinked_img.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and img_shrinked is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, img_shrinked)
    img_enlarged = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
    img_enlarged = face_box(img_enlarged)
    enlargedimg_name = f'{image_name}_enlarged_img.jpeg'
    if not os.path.isfile(os.path.join(path, enlargedimg_name)) and img_enlarged is not None:
      save_path = os.path.join(path, enlargedimg_name)
      cv2.imwrite(save_path, img_enlarged)


def adjust_lighting(image, path, image_name, brightness=None, contrast=None):
    if brightness is None or contrast is None:
      brightness = random.randint(-150, 150)/5.0
      contrast = round(random.uniform(1, 2), 2)
    adjusted_image = cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness)
    adjusted_image = face_box(adjusted_image)
    img_name = f'{image_name}_brightness_{brightness}_contrast_{contrast}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and adjusted_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, adjusted_image)

def adjust_contrast(image, value, path, image_name):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    lab[:, :, 0] = np.clip(lab[:, :, 0] * value, 0, 255)
    adjusted_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    adjusted_image = face_box(adjusted_image)
    img_name = f'{image_name}_contrast_value{value}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and adjusted_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, adjusted_image)

def adjust_saturation(image, value, path, image_name):
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    hls[:, :, 2] += value
    hls[:, :, 2] = np.clip(hls[:, :, 2], 0, 255)
    adjusted_image = cv2.cvtColor(hls, cv2.COLOR_HLS2BGR)
    adjusted_image = face_box(adjusted_image)
    img_name = f'{image_name}_saturation_value{value}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and adjusted_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, adjusted_image)

def flip_image(image, path, image_name):
  flipped_img = cv2.flip(image,1)
  flipped_img = face_box(flipped_img)
  flipimg_name = f'{image_name}_flipped.jpeg'
  if not os.path.isfile(os.path.join(path, flipimg_name)) and flipped_img is not None:
    save_path = os.path.join(path, flipimg_name)
    cv2.imwrite(save_path, flipped_img)

def face_box(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces_detected = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)

    largest_face_area = 0
    largest_face = None

    for (x, y, w, h) in faces_detected:
        face_area = w * h
        if face_area > largest_face_area:
            largest_face_area = face_area
            largest_face = (x, y, w, h)
    if largest_face is not None:
        x, y, w, h = largest_face
        largest_face_image = image[y:y+h, x:x+w]
        resized_largest_face_image = cv2.resize(largest_face_image, (200, 200))
        return resized_largest_face_image
    else:
        return None

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
f = open(validate_path + "file_mapping.txt", "r")
next(f)
for line in f:
  image_name, label = line.split(" ")
  print(image_name, label)
  image = cv2.imread(validate_path + image_name)
  validate_label_path = os.path.join(validate_path, label)
  if not os.path.isdir(validate_label_path):
    os.mkdir(validate_label_path)
  if not os.path.isfile(os.path.join(validate_label_path, image_name)) and image is not None:
      face_cropped_image = face_box(image)
      save_path = os.path.join(validate_label_path, image_name)
      if face_cropped_image is not None:
        cv2.imwrite(save_path, face_cropped_image)
  #translation
  for dx, dy in [(25, 20), (-25, -20)]:
      translate_image(image, dx, dy, validate_label_path, image_name)
  #flipping
  flip_image(image, validate_label_path, image_name)
  #rotation
  for angle in [7, -7, 10, -10]:
      rotate_image(image, validate_label_path, image_name, angle)
  #background-lighting
  brightness = 3
  contrast = 1.6
  bc_image = cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness)
  bc_image = face_box(bc_image)
  bkgAdimg_name = f'{image_name}_backgroundAdj.jpeg'
  if not os.path.isfile(os.path.join(validate_label_path, bkgAdimg_name)) and bc_image is not None:
    save_path = os.path.join(validate_label_path, bkgAdimg_name)
    cv2.imwrite(save_path, bc_image)
  #scaling
  scale_image(image, validate_label_path, 1.5, image_name)
  #brightness
  for value in [-18, 38, 50]:
    adjust_lighting(image, validate_label_path, image_name, value, 1)
  #contrast
  for value in [1.3, 1.5, 1.7]:
    adjust_contrast(image, value, validate_label_path, image_name)
  # Adjust saturation
  for value in [50, 60, 70]:
    adjust_saturation(image, value, validate_label_path, image_name)
  #sheared-y
  rows, cols = image.shape[:2]
  M = np.float32([[1,   0, 0], [0.5, 1, 0], [0,   0, 1]])
  sheared_yimg = cv2.warpPerspective(image, M, (int(cols*1.5), int(rows*1.5)))
  sheared_yimg = face_box(sheared_yimg)
  shearyimg_name = f'{image_name}_sheared_yimg.jpeg'
  if not os.path.isfile(os.path.join(validate_label_path, shearyimg_name)) and sheared_yimg is not None:
    save_path = os.path.join(validate_label_path, shearyimg_name)
    cv2.imwrite(save_path, sheared_yimg)
  #sheared-x
  Ma = np.float32([[1, 0.5, 0], [0, 1, 0], [0, 0, 1]])
  sheared_ximg = cv2.warpPerspective(image, Ma, (int(cols*1.5), int(rows*1.5)))
  sheared_ximg = face_box(sheared_ximg)
  shearximg_name = f'{image_name}_sheared_ximg.jpeg'
  if not os.path.isfile(os.path.join(validate_label_path, shearximg_name)) and sheared_ximg is not None:
    save_path = os.path.join(validate_label_path, shearximg_name)
    cv2.imwrite(save_path, sheared_ximg)
  print("Generated all images")

0206_1.jpeg sampagaonrahul

Generated all images
0206_2.jpeg sivarajusairevanth

Generated all images
0206_3.jpeg pereiranerissagodfrey

Generated all images
0206_4.jpeg wukaiyue

Generated all images
0206_5.jpeg mendonakshay

Generated all images
0206_6.jpeg zhangyuanzhen

Generated all images
0206_7.jpeg manglaniroshanlakhi

Generated all images
0206_8.jpeg negiparth

Generated all images
0206_9.jpeg banmingkai

Generated all images
0206_10.jpeg yashasvi

Generated all images
0206_11.jpeg liuhongji

Generated all images
0206_12.jpeg zuluagagonzalezisabel

Generated all images
0206_13.jpeg chenziang

Generated all images
0206_14.jpeg chientingwei

Generated all images
0206_15.jpeg gowdarachandrashekarappasrivarsha

Generated all images
0206_16.jpeg lozanoroberto

Generated all images
0206_17.jpeg vennavellirajashekarreddy

Generated all images
0206_18.jpeg selinayu

Generated all images
0206_19.jpeg lishumeng

Generated all images
0206_20.jpeg perambuduruvishnu

Generated all images
0

ValueError: too many values to unpack (expected 2)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
f = open(train_path + "file_mapping.txt", "r")
next(f)
for line in f:
  image_name, label = line.split(" ")
  if label != "shahmanali": continue
  print(image_name, label)
  image = cv2.imread(train_path + image_name)
  train_label_path = os.path.join(train_path, label)
  if not os.path.isdir(train_label_path):
    os.mkdir(train_label_path)
  if not os.path.isfile(os.path.join(train_label_path, image_name)) and image is not None:
      face_cropped_image = face_box(image)
      save_path = os.path.join(train_label_path, image_name)
      if face_cropped_image is not None:
        cv2.imwrite(save_path, face_cropped_image)
  #translation
  for dx, dy in [(25, 20), (-25, -20)]:
      translate_image(image, dx, dy, train_label_path, image_name)
  #flipping
  flip_image(image, train_label_path, image_name)
  #rotation
  for angle in [7, -7, 10, -10]:
      rotate_image(image, train_label_path, image_name, angle)
  #background-lighting
  brightness = 3
  contrast = 1.6
  bc_image = cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness)
  bc_image = face_box(bc_image)
  bkgAdimg_name = f'{image_name}_backgroundAdj.jpeg'
  if not os.path.isfile(os.path.join(train_label_path, bkgAdimg_name)) and bc_image is not None:
    save_path = os.path.join(train_label_path, bkgAdimg_name)
    cv2.imwrite(save_path, bc_image)
  #scaling
  scale_image(image, train_label_path, 1.5, image_name)
  #brightness
  for value in [-18, 38, 50]:
    adjust_lighting(image, train_label_path, image_name, value, 1)
  #contrast
  for value in [1.3, 1.5, 1.7]:
    adjust_contrast(image, value, train_label_path, image_name)
  # Adjust saturation
  for value in [50, 60, 70]:
    adjust_saturation(image, value, train_label_path, image_name)
  #sheared-y
  rows, cols = image.shape[:2]
  M = np.float32([[1,   0, 0], [0.5, 1, 0], [0,   0, 1]])
  sheared_yimg = cv2.warpPerspective(image, M, (int(cols*1.5), int(rows*1.5)))
  sheared_yimg = face_box(sheared_yimg)
  shearyimg_name = f'{image_name}_sheared_yimg.jpeg'
  if not os.path.isfile(os.path.join(train_label_path, shearyimg_name)) and sheared_yimg is not None:
    save_path = os.path.join(train_label_path, shearyimg_name)
    cv2.imwrite(save_path, sheared_yimg)
  #sheared-x
  Ma = np.float32([[1, 0.5, 0], [0, 1, 0], [0, 0, 1]])
  sheared_ximg = cv2.warpPerspective(image, Ma, (int(cols*1.5), int(rows*1.5)))
  sheared_ximg = face_box(sheared_ximg)
  shearximg_name = f'{image_name}_sheared_ximg.jpeg'
  if not os.path.isfile(os.path.join(train_label_path, shearximg_name)) and sheared_ximg is not None:
    save_path = os.path.join(train_label_path, shearximg_name)
    cv2.imwrite(save_path, sheared_ximg)
  print("Generated all images")

0123_1.jpeg wukaiyue

Generated all images
0123_2.jpeg vanderlindenilona

Generated all images
0123_3.jpeg zhangyuanzhen

Generated all images
0123_4.jpeg liuhongji

Generated all images
0123_5.jpeg lozanoroberto

Generated all images
0123_6.jpeg yashasvi

Generated all images
0123_7.jpeg huangjiaoyan

Generated all images
0123_8.jpeg manglaniroshanlakhi

Generated all images
0123_9.jpeg somaniachal

Generated all images
0123_10.jpeg zotaharsh

Generated all images
0123_11.jpeg pereiranerissagodfrey

Generated all images
0123_12.jpeg zuluagagonzalezisabel

Generated all images
0123_13.jpeg lishumeng

Generated all images
0123_14.jpeg ravijayanthidhanasekar

Generated all images
0123_15.jpeg oraisisaac

Generated all images
0123_16.jpeg kodipunzulanandini

Generated all images
0123_17.jpeg upadhyevaishnavi

Generated all images
0123_18.jpeg chientingwei

Generated all images
0123_19.jpeg amarisian

Generated all images
0123_20.jpeg zhouchuandi

Generated all images
0123_21.jpeg vennavel

ValueError: too many values to unpack (expected 2)